In [1]:
import pandas as pd
import numpy as np

In [111]:
data_path = './data/'
df_train = pd.read_csv(data_path + 'train.csv')
df_test = pd.read_csv(data_path + 'test.csv')

train_lable = df_train['Survived']
df_train = df_train.drop(['Survived'] , axis=1)

df = pd.concat([df_train,df_test])
print(df.shape)
df.head()

(1309, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [112]:
# Pcalss 無遺失值
print(np.where(pd.isnull(df[['Pclass']])))

(array([], dtype=int64), array([], dtype=int64))


In [113]:
# Name 無遺失值
print(np.where(pd.isnull(df[['Name']])))

(array([], dtype=int64), array([], dtype=int64))


In [114]:
# 使用title作為feature
df['title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
print(df['title'].unique())
df['title'].head()

['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'Countess' 'Jonkheer' 'Dona']


0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
Name: title, dtype: object

In [115]:
# Ticket 無遺失值
print(np.where(pd.isnull(df[['Ticket']])))
ti_cnt = df.groupby(['Ticket'])['PassengerId'].agg({'ticket_cnt':'count'}).reset_index()
df = df.merge(ti_cnt, left_on='Ticket', right_on='Ticket').sort_values('PassengerId')

(array([], dtype=int64), array([], dtype=int64))


D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [116]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title,ticket_cnt
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
3,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
4,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
6,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


In [117]:
# Sex 無遺失值
print(np.where(pd.isnull(df[['Sex']])))

(array([], dtype=int64), array([], dtype=int64))


In [118]:
# Sibsp, Parch無遺失值
print(np.where(pd.isnull(df[['SibSp']])))
print(np.where(pd.isnull(df[['Parch']])))

df['Family'] = df['SibSp'] + df['Parch']

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [119]:
# Age 有遺失值須進行補值
print(np.where(pd.isnull(df[['Age']])))

(array([   5,   17,   19,   26,   28,   29,   31,   32,   36,   42,   45,
         46,   47,   48,   55,   64,   65,   76,   77,   82,   87,   95,
        101,  107,  109,  121,  126,  128,  140,  154,  158,  159,  166,
        168,  176,  180,  181,  185,  186,  196,  198,  201,  214,  223,
        229,  235,  240,  241,  250,  256,  260,  264,  270,  274,  277,
        284,  295,  298,  300,  301,  303,  304,  306,  324,  330,  334,
        335,  347,  351,  354,  358,  359,  364,  367,  368,  375,  384,
        388,  409,  410,  411,  413,  415,  420,  425,  428,  431,  444,
        451,  454,  457,  459,  464,  466,  468,  470,  475,  481,  485,
        490,  495,  497,  502,  507,  511,  517,  522,  524,  527,  531,
        533,  538,  547,  552,  557,  560,  563,  564,  568,  573,  578,
        584,  589,  593,  596,  598,  601,  602,  611,  612,  613,  629,
        633,  639,  643,  648,  650,  653,  656,  667,  669,  674,  680,
        692,  697,  709,  711,  718,  727,  732,  

In [120]:
# 用 Pcalss, title, Sex, Sibsp, Parch建立迴歸模型來預測 Age
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
df['Pclass'] = df['Pclass'].astype('category')
df_temp =  pd.get_dummies(df[['Pclass','title','Sex']])
df_temp['SibSp'] = df['SibSp']
df_temp['Parch'] = df['Parch']
df_temp['Age'] = df['Age']

df_temp_train = df_temp[df_temp[['Age']].isnull().values==False]
df_temp_value = df_temp_train['Age']
df_temp_train = df_temp_train.drop(['Age'], axis=1)

lasso = linear_model.Lasso(alpha=0.01)
lasso.fit(df_temp_train, df_temp_value)

df_temp_test = df_temp[df_temp[['Age']].isnull().values==True]
df_temp_test = df_temp_test.drop(['Age'], axis=1)
y_pred = list(lasso.predict(df_temp_test))
y_pred = [ round(elem, 0) for elem in y_pred ]

df.Age[df.Age.isna()] = y_pred

D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [121]:
print(np.where(pd.isnull(df[['Fare']])))

df['Fare'].fillna(df['Fare'].median(), inplace=True)

(array([1043], dtype=int64), array([0], dtype=int64))


In [122]:
# 針對Fare做標準化
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df['Fare_stand'] = StandardScaler().fit_transform(df[['Fare']])
df['Fare_minmax'] = MinMaxScaler().fit_transform(df[['Fare']])
df['Fare_log'] = np.log(df[['Fare']]+1)


In [123]:
# 補缺失的 Embarked
print(np.where(pd.isnull(df[['Embarked']])))
df.Embarked[df.Embarked.isna()] = df['Embarked'].mode()[0]

(array([ 61, 829], dtype=int64), array([0, 0], dtype=int64))


D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [124]:
#  針對Cabin，並只取第一個字母
df['Cabin'].fillna("-1", inplace=True)
df['Cabin_class'] = df['Cabin'].str.get(0)

In [125]:
# 使用 Embarked, Fare, Pcalss, title, Sex, Sibsp, Parch補 Cabin
'''
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df_temp =  pd.get_dummies(df[['Pclass','title','Sex','Embarked']])
df_temp['SibSp'] = df['SibSp']
df_temp['Parch'] = df['Parch']
df_temp['Fare'] = df['Fare']

df_temp['Cabin_class'] = df['Cabin_class']

df_temp_train = df_temp[df_temp.Cabin_class != '-']
df_temp_value = df_temp_train['Cabin_class']
df_temp_train = df_temp_train.drop(['Cabin_class'], axis=1)

clf = RandomForestClassifier(n_estimators=20, max_depth=4)
clf.fit(df_temp_train, df_temp_value)


df_temp_test = df_temp[df_temp.Cabin_class == '-']
df_temp_test = df_temp_test.drop(['Cabin_class'], axis=1)
y_pred = list(clf.predict(df_temp_test))

df.Cabin_class[df.Cabin_class == '-'] = y_pred
'''

"\nfrom sklearn.preprocessing import OneHotEncoder\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error\n\ndf_temp =  pd.get_dummies(df[['Pclass','title','Sex','Embarked']])\ndf_temp['SibSp'] = df['SibSp']\ndf_temp['Parch'] = df['Parch']\ndf_temp['Fare'] = df['Fare']\n\ndf_temp['Cabin_class'] = df['Cabin_class']\n\ndf_temp_train = df_temp[df_temp.Cabin_class != '-']\ndf_temp_value = df_temp_train['Cabin_class']\ndf_temp_train = df_temp_train.drop(['Cabin_class'], axis=1)\n\nclf = RandomForestClassifier(n_estimators=20, max_depth=4)\nclf.fit(df_temp_train, df_temp_value)\n\n\ndf_temp_test = df_temp[df_temp.Cabin_class == '-']\ndf_temp_test = df_temp_test.drop(['Cabin_class'], axis=1)\ny_pred = list(clf.predict(df_temp_test))\n\ndf.Cabin_class[df.Cabin_class == '-'] = y_pred\n"

In [126]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [127]:
df_temp =  pd.get_dummies(df[['Pclass', 'title', 'Sex', 'Embarked', 'Cabin_class']])

In [128]:
df2 = pd.concat([df, df_temp], axis=1)
df2 = df2.drop(['Pclass', 'title', 'Sex', 'Embarked', 'Cabin_class'], axis=1)

In [129]:
train_data = df2[:df_train.shape[0]]
test_data = df2[df_train.shape[0]:]

In [130]:
train_data.to_pickle('./data/train_data.pkl')
test_data.to_pickle('./data/test_data.pkl')
train_lable.to_pickle('./data/train_lable.pkl')